# Parsing latest version of LAPD eblotter email

In [163]:
import pandas as pd

In [164]:
df = pd.read_excel('input/dabis_eblotter.xlsx', skiprows=range(0,3))

In [165]:
df.iloc[1,1] + ' ' + df.iloc[1,2]

'11/07/19 00:05'

In [170]:
df.iloc[i,0]

5792555

In [90]:
arrests['released_date'] = pd.to_datetime((arrests['REL_DT'] + ' ' + \
                          arrests['REL_TM'])).apply(lambda x: \
                                                    x.strftime('%Y-%m-%d %H:%M')if not pd.isnull(x) else '')

In [91]:
arrests['booking_date'] = pd.to_datetime((arrests['REL_DT'] + ' ' + \
                          arrests['REL_TM'])).apply(lambda x: \
                                                    x.strftime('%Y-%m-%d %H:%M')if not pd.isnull(x) else '')

In [95]:
# Format dates for Django
def parse_booking_date(self, date_value, time_value):
    if time_value == "24:00":
        time_value = "23:59"
    value = "{} {}".format(date_value, time_value).strip()
    if value:
        return datetime.datetime.strptime(value, "%m/%d/%y %H:%M")
    return None

def parse_released_date(self, date_value, time_value):
    if time_value == "24:00":
        time_value = "23:59"
    value = "{} {}".format(date_value, time_value).strip()
    if value:
        return datetime.datetime.strptime(value, "%m/%d/%y %H:%M")
    return None

def to_float(self, value):
    if value == "NO BAIL":
        return None
    elif value:
        try:
            return float(value)
        except ValueError:
            return None
    return value or None

def to_int(self, value):
    if value == "NO BAIL":
        return None
    elif value:
        return int(value)
    return value or None

def to_address(self, add):
    add_clean = add[:-2].strip()
    place = re.split("\s+\s+", add_clean)
    if place:
        return " ".join(place[:-1])
    return ""

def to_hometown(self, add):
    add_clean = add[:-2].strip()
    place = re.split("\s+\s+", add_clean)
    if place:
        return "".join(place[-1:])
    return ""

def to_state(self, add):
    state = add[-2:]
    if state:
        return state
    return ""

In [ ]:
charge_list = [
    dict(
        raw_code=get_cell(19),
        level=get_cell(20),
        bail_amount=self.to_float(get_cell(21)),
    ),
    dict(
        raw_code=get_cell(23),
        level=get_cell(24),
        bail_amount=self.to_float(get_cell(25)),
    ),
    dict(
        raw_code=get_cell(26),
        level=get_cell(27),
        bail_amount=self.to_float(get_cell(28)),
    )
]

In [107]:
arrests.iloc[0]

BKG_NUM                                                   5792555
BKG_DT                                                   11/07/19
BKG_TM                                                      00:03
ARR_DIV                                                      4265
ARR_LOC                                                       OSO
ARR_CRSST                                                 SATICOY
ARR_ZIP                                                       NaN
LAST_NM                                                  GONZALEZ
FRST_NM                                                   NATALIE
MID_NM                                                       ROSE
OCCUPATION                                            DOG GROOMER
SEX                                                             F
DESC                                                            H
HAIR                                                          BLK
EYES                                                          BRO
HGT       

In [97]:
# arrests['arrestee_address'] = arrests.RES_LOC.apply(to_address)
# arrests['arrestee_city'] = arrests.RES_LOC.apply(to_hometown) 
# arrests['arrestee_state'] = arrests.RES_LOC.apply(to_state) 
arrests['total_bail'] = arrests.TOT_BAIL.apply(to_float) 

TypeError: to_float() missing 1 required positional argument: 'value'

In [9]:
arrests.rename = [
    'BKG_NUM':'booking_num',
    'booking_date': 'booking_date',
#     'BKG_DT':'',
#     'BKG_TM':'',
    'ARR_DIV':'arr_div',
    'ARR_LOC':'arrest_address',
    'ARR_CRSST':'arrest_cross_street',
    'ARR_ZIP':'arrest_zipcode',
    'LAST_NM':'last_name',
    'FRST_NM':'first_name',
    'MID_NM':'middle_name',
    'OCCUPATION':'occupation',
    'SEX':'gender',
    'DESC':'race',
    'HAIR':'hair_color',
    'EYES':'eye_color',
    'HGT':'height',
    'WGT':'weight',
    'DOB':'date_of_birth',
    'RES_LOC':'',
    'CHARGE':'',
    'CHG_LVL':'',
    'BAIL':'',
    'CHG_DESC':'',
    'CHARGE.1':'',
    'CHG_LVL.1':'',
    'BAIL.1':'',
    'CHARGE.2':'',
    'CHG_LVL.2':'',
    'BAIL.2':'',
#     'TOT_BAIL':'',
    'ARR_DT':'',
    'ARR_TM':'',
#     'REL_DT':'',
#     'REL_TM':'',
    'REL_REASON':'release_reason',
    'REL_LOC':'release_location',
]

In [98]:
arrests.dtypes

BKG_NUM                   int64
BKG_DT                   object
BKG_TM                   object
ARR_DIV                  object
ARR_LOC                  object
ARR_CRSST                object
ARR_ZIP                 float64
LAST_NM                  object
FRST_NM                  object
MID_NM                   object
OCCUPATION               object
SEX                      object
DESC                     object
HAIR                     object
EYES                     object
HGT                       int64
WGT                       int64
DOB              datetime64[ns]
RES_LOC                  object
CHARGE                   object
CHG_LVL                  object
BAIL                     object
CHG_DESC                 object
CHARGE.1                 object
CHG_LVL.1                object
BAIL.1                   object
CHARGE.2                 object
CHG_LVL.2                object
BAIL.2                  float64
TOT_BAIL                 object
ARR_DT                   object
ARR_TM  

In [ ]:
new_arrest_record = Arrest.objects.create(
                attachment=self,
#                 booking_num=self.to_int(booking_num),
                booking_date=self.parse_date(get_cell(1), get_cell(2)),
#                 arr_div=get_cell(3),
                arrest_date=self.parse_date(get_cell(30), get_cell(31)),
#                 arrest_address=get_cell(4),
#                 arrest_cross_street=get_cell(5),
#                 arrest_zipcode=self.to_int(get_cell(6)),
#                 first_name=get_cell(8),
#                 middle_name=get_cell(9),
#                 last_name=get_cell(7),
#                 occupation=get_cell(10),
#                 gender=get_cell(11),
#                 race=get_cell(12),
#                 hair_color=get_cell(13),
#                 eye_color=get_cell(14),
#                 height=self.to_int(get_cell(15)),
#                 weight=self.to_int(get_cell(16)),
#                 date_of_birth=xldate_as_datetime(self.to_int(get_cell(17)), None),
#                 arrestee_address=self.to_address(get_cell(18)),
#                 arrestee_city=self.to_hometown(get_cell(18)),
#                 arrestee_state=self.to_state(get_cell(18)),
#                 total_bail=self.to_float(get_cell(29)),
                release_date=self.parse_date(get_cell(32), get_cell(33)),
                release_reason=get_cell(34),
                release_location=get_cell(35),
              )

In [6]:
arrests.columns

Index(['BKG_NUM', 'BKG_DT', 'BKG_TM', 'ARR_DIV', 'ARR_LOC', 'ARR_CRSST',
       'ARR_ZIP', 'LAST_NM', 'FRST_NM', 'MID_NM', 'OCCUPATION', 'SEX', 'DESC',
       'HAIR', 'EYES', 'HGT', 'WGT', 'DOB', 'RES_LOC', 'CHARGE', 'CHG_LVL',
       'BAIL', 'CHG_DESC', 'CHARGE.1', 'CHG_LVL.1', 'BAIL.1', 'CHARGE.2',
       'CHG_LVL.2', 'BAIL.2', 'TOT_BAIL', 'ARR_DT', 'ARR_TM', 'REL_DT',
       'REL_TM', 'REL_REASON', 'REL_LOC'],
      dtype='object')

In [4]:
# columns to match the datadesk model (but missing the arrestee address)

arrests.rename(columns={
    'bkg_num':'booking_num',
    'bkg_num':'booking_date',
    'arr_div':'arr_div',
    'arr_dt':'arrest_date',
    'arr_loc':'arrest_address',
    'arr_crsst':'arrest_cross_street',
    'arr_zip':'arrest_zipcode',
    'frst_nm':'first_name',
    'mid_nm':'middle_name',
    'last_nm':'last_name',
    'occupation':'occupation',
    'sex':'gender',
    'desc':'race',
    'hair':'hair_color',
    'eyes':'eye_color',
    'hgt':'height',
    'wgt':'weight',
    'dob':'date_of_birth',
    'tot_bail':'total_bail',
    'rel_dt':'release_date',
    'rel_reason':'release_reason',
    'res_loc':'release_location',}, inplace=True)

In [5]:
arrests.head()

,BKG_NUM,BKG_DT,BKG_TM,ARR_DIV,ARR_LOC,ARR_CRSST,ARR_ZIP,LAST_NM,FRST_NM,MID_NM,...,CHARGE.2,CHG_LVL.2,BAIL.2,TOT_BAIL,ARR_DT,ARR_TM,REL_DT,REL_TM,REL_REASON,REL_LOC
0,5792555,11/07/19,00:03,4265,OSO,SATICOY,NaN,GONZALEZ,NATALIE,ROSE,...,602(K)PC,M,5000.0,110000.00,11/06/19,21:40,NaN,NaN,NaN,NaN
1,5792556,11/07/19,00:05,4212,51ST,FIGUEROA,NaN,LOVE,JOSHUA,NaN,...,NaN,NaN,NaN,25000.00,11/06/19,20:00,NaN,NaN,NaN,NaN
2,5792557,11/07/19,00:05,4203,23RD ST,FIGUEROA ST,NaN,ANTUNES,JOSE,NaN,...,NaN,NaN,NaN,20000.00,11/06/19,22:00,NaN,NaN,NaN,NaN
3,5792560,11/07/19,00:12,4206,1655 N WESTERN AV,NaN,90028.0,GULLEY,BRIANNA,NaN,...,NaN,NaN,NaN,2000.00,11/06/19,22:30,11/07/19,04:37,OR,4206.0
4,5792564,11/07/19,00:13,4206,1828 WILCOX,NaN,NaN,SIMENTAL,ELIZABETH,NaN,...,NaN,NaN,NaN,2000.00,11/06/19,21:45,11/07/19,00:53,OR,4206.0
